In [1]:
# Source: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews

In [2]:
import pandas as pd
import numpy as np

books = pd.read_csv("./raw_books_data.csv")

In [3]:
books.columns

Index(['Title', 'description', 'authors', 'image', 'previewLink', 'publisher',
       'publishedDate', 'infoLink', 'categories', 'ratingsCount'],
      dtype='object')

In [4]:
filtered_book = books
filtered_book = filtered_book[~filtered_book["categories"].isna()]
filtered_book = filtered_book[~filtered_book["description"].isna()]
filtered_book = filtered_book[~filtered_book["Title"].isna()]
filtered_book = filtered_book[~filtered_book["publishedDate"].isna()]

In [5]:
filtered_book["categories_list"] = filtered_book["categories"].apply(
    lambda x: eval(x) if type(x) == str else []
)

filtered_book["categories_list"] = (
    filtered_book["categories_list"]
    .apply(lambda x: [c.split("&") for c in x])
    .apply(lambda x: np.concatenate(x))
    .apply(lambda x: [c.split(", ") for c in x])
    .apply(lambda x: np.concatenate(x).tolist())
    .apply(lambda x: [c.strip() for c in x])
)

In [6]:
flattened_values = [item for sublist in filtered_book['categories_list'] for item in sublist]

# Convert the flattened values to a Series and get the value counts (distribution)
distribution = pd.Series(flattened_values).value_counts()

# Display the distribution of unique values
categories_df = distribution[distribution > 100] #.to_csv('categories.csv')

In [7]:
categories = categories_df.index.to_list()

In [8]:
filtered_book = filtered_book[filtered_book["categories_list"].apply(lambda x: np.array([c in categories for c in x]).any())]
filtered_book = filtered_book.drop(columns=['previewLink', 'image', 'categories', 'infoLink', 'ratingsCount'])

In [9]:
filtered_book["book_id"] = filtered_book.index

In [10]:
ratings = pd.read_csv("./raw_books_rating.csv")


In [11]:
len(ratings)

3000000

In [12]:
ratings.columns

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')

In [13]:
ratings_filtered = ratings[["Id", "Title", "User_id", "review/score"]]
ratings_filtered = ratings_filtered[~ratings_filtered["User_id"].isna()]

In [14]:
filtered_book["Title"].unique().__len__()

124440

In [15]:
merged_data = ratings_filtered.merge(filtered_book, on="Title")
merged_data['count'] = merged_data.groupby('Title')['Title'].transform('count')
merged_data['vote_average'] = merged_data.groupby('Title')['review/score'].transform('mean')
merged_data.drop(columns=["Id"], inplace=True)
merged_data.rename(columns={'Title': 'title', "User_id": "user_id", "publishedDate": "published_year", "review/score": "rating", "categories_list": "categories"}, inplace=True)
merged_data["published_year"] = merged_data["published_year"].apply(lambda x: x.split("-")[0])
merged_data = merged_data[merged_data["count"] >= 5]

In [18]:
post_processed_books = merged_data[["title", "description", "authors", "publisher", "published_year", "categories", "book_id", "count", "vote_average"]]
post_processed_books.drop_duplicates(subset=['book_id'], inplace=True)

/tmp/ipykernel_116744/1466325062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_processed_books.drop_duplicates(subset=['book_id'], inplace=True)


In [19]:
post_processsed_ratings = merged_data[["user_id", "book_id", "rating"]]

In [20]:
post_processed_books.head()

,title,description,authors,publisher,published_year,categories,book_id,count,vote_average
0,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,2005,"[Biography, Autobiography]",1,9,4.555556
13,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],iUniverse,2005,[Fiction],3,32,3.718750
53,Dramatica for Screenwriters,Dramatica for Screenwriters by Armando Saldana...,['Armando Salda A-Mora'],NaN,2005,[Reference],11,10,4.700000
68,The Ultimate Guide to Law School Admission: In...,This collection brings together a distinguishe...,['Fiona Cownie'],Bloomsbury Publishing,2010,[Law],16,6,3.333333
77,Alaska Sourdough,"""Sourdough is a magical food"", as author Ruth ...",['Ruth Allman'],Alaska Northwest Books,1976,[Cooking],19,25,4.280000


In [21]:
post_processsed_ratings.head()

,user_id,book_id,rating
0,A30TK6U7DNS82R,1,5.0
1,A3UH4UZ4RSVO82,1,5.0
2,A2MVUWT453QH61,1,4.0
3,A22X4XUPKF66MR,1,4.0
4,A2F6NONFUDB6UK,1,4.0


In [22]:
post_processsed_ratings.describe()

,book_id,rating
count,1.513047e+06,1.513047e+06
mean,9.318937e+04,4.218488e+00
std,6.225811e+04,1.188469e+00
min,1.000000e+00,1.000000e+00
25%,3.411900e+04,4.000000e+00
50%,9.059600e+04,5.000000e+00
75%,1.470680e+05,5.000000e+00
max,2.123990e+05,5.000000e+00


In [23]:
post_processed_books.to_csv('postprocessed_books.csv', index=False)
post_processsed_ratings.to_csv('postprocessed_ratings.csv', index=False)